In [1]:
import os
import numpy as np
from pathlib import Path

from utils import BOSS_randoms, rh5_hodcatalog, fiducial_cosmology
from nbodykit.lab import transform

In [2]:
input_dir = f'/mnt/home/bregaldosaintblancard/ceph/simbig/tokyo/'

In [3]:
dataset = 'train' # 'train' or 'test'

In [4]:
input_train_dir = os.path.join(input_dir, dataset)
train_files = sorted(os.listdir(input_train_dir))
params = np.loadtxt(os.path.join(input_dir, f'params_{dataset}.txt'))

In [5]:
params_ids = params[:, :2].astype(int)
params_thetas = params[:, 2:]

In [6]:
sim_id = 0

i, j = params_ids[sim_id]
theta = params_thetas[sim_id]
print("Sim ID: ", sim_id)
print("Quijote index: ", i)
print("HOD index: ", j)
print("Parameters: ", theta)

# There are 14 parameters ordered as follows:
# 5 cosmological parameters: Omega_m, Omega_b, h, n_s, sigma_8
# 9 cosmological parameters: log_M_min, sigma_log_M, log_M_0, log_M_1, alpha, A_bias, eta_conc, eta_cen, eta_sat
# For more details see arXiv:2211.00723 and arXiv:2211.00660

Sim ID:  0
Quijote index:  8
HOD index:  0
Parameters:  [ 3.21500000e-01  5.48700000e-02  6.42900000e-01  1.05310000e+00
  7.50900000e-01  1.37224233e+01  3.19915407e-01  1.35114904e+01
  1.46053790e+01  7.16793007e-01 -4.12212926e-03  1.74206449e+00
  2.93004618e-01  1.43023682e+00]


In [7]:
fgal = f'hod.quijote_LH{i}.z0p5.cmass_sgc.v4.{j}.hdf5' if dataset == 'train' else f'hod.quijote_fid{i}.z0p5.cmass_sgc.v4.{j}.hdf5'

# read galaxy catalog
gals = rh5_hodcatalog(os.path.join(input_train_dir, fgal))

# construct corresponding randoms
rand = BOSS_randoms(gals, input_dir)

In [8]:
# RA, DEC, Z -> 3D
cosmo = fiducial_cosmology()
galaxy_positions = transform.SkyToCartesian(gals['RA'], gals['DEC'], gals['Z'], cosmo=cosmo)
randoms_positions = transform.SkyToCartesian(rand['RA'], rand['DEC'], rand['Z'], cosmo=cosmo)

In [9]:
# For conversion to numpy arrays (if needed)
galaxy_positions = galaxy_positions.compute()
randoms_positions = randoms_positions.compute()